<h1>Préparation du dataset

In [1]:
from tensorflow import *
import tensorflow.keras as krs
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn import *
import os 


In [2]:
path_real='F:\\PA\\transactions_horodateur\\capacites_horodateurs_avec_geoloc.csv'
file_real_place=pd.read_csv(path_real,sep=';')


In [2]:
def prepare_csv(file):
    df=file
    #del(df['date'])
    #create 6 days of week instead of column day_of_week
    #df=pd.concat([df,pd.get_dummies(df["day_of_week"],prefix="jour")],axis=1)
    #store only label to dummy:
    col=df.columns.drop(labels=['horodateur','Longitude','Latitude','day_of_week'])
    tranche = ['00:00:00-00:30:00', '00:30:00-01:00:00', '01:00:00-01:30:00', '01:30:00-02:00:00', '02:00:00-02:30:00',
           '02:30:00-03:00:00', '03:00:00-03:30:00', '03:30:00-04:00:00', '04:00:00-04:30:00', '04:30:00-05:00:00',
           '05:00:00-05:30:00', '05:30:00-06:00:00', '06:00:00-06:30:00', '06:30:00-07:00:00', '07:00:00-07:30:00',
           '07:30:00-08:00:00', '08:00:00-08:30:00', '08:30:00-09:00:00', '09:00:00-09:30:00', '09:30:00-10:00:00',
           '10:00:00-10:30:00', '10:30:00-11:00:00', '11:00:00-11:30:00', '11:30:00-12:00:00', '12:00:00-12:30:00',
           '12:30:00-13:00:00', '13:00:00-13:30:00', '13:30:00-14:00:00', '14:00:00-14:30:00', '14:30:00-15:00:00',
           '15:00:00-15:30:00', '15:30:00-16:00:00', '16:00:00-16:30:00', '16:30:00-17:00:00', '17:00:00-17:30:00',
           '17:30:00-18:00:00', '18:00:00-18:30:00', '18:30:00-19:00:00', '19:00:00-19:30:00', '19:30:00-20:00:00',
           '20:00:00-20:30:00', '20:30:00-21:00:00', '21:00:00-21:30:00', '21:30:00-22:00:00', '22:00:00-22:30:00',
           '22:30:00-23:00:00', '23:00:00-23:30:00', '23:30:00-00:00:00']
    df_horo_day = df[['Longitude','Latitude', "day_of_week"]]
    df_horo_day = pd.concat([df_horo_day, pd.get_dummies(df_horo_day['day_of_week'], prefix="day",dtype=int)], axis=1)
    del df_horo_day["day_of_week"]

    df_horo_day_last = df_horo_day.iloc[0:0]
    for index, row in df_horo_day.iterrows():
        df_horo_day_last = df_horo_day_last.append([row] * 48, ignore_index=True)
    
    test = pd.DataFrame(np.identity(48), columns=tranche)
    test = test.astype(int)
    test = test.append([test] * (len(df_horo_day) - 1), ignore_index=True)
    df_horo_day_last[test.columns.tolist()] = test
    X = df_horo_day_last
    
    tmp=df.filter(tranche).values.flatten()
    Y=pd.DataFrame(tmp.reshape(len(tmp),1),columns=['target'])
    #Y = df.filter(tranche).values.flatten()
    return pd.concat([X,Y],axis=1)
    

In [3]:
path='F:\\PA\\transactions_horodateur\\dataset_final\\final\\'
files=os.listdir(path)

In [7]:
#open 15 files:
rng=0
cols=prepare_csv(pd.read_csv(path+files[0],sep=';')).columns
first_file=pd.DataFrame([],columns=cols)

for e in files:
    x1=pd.read_csv(path+e,sep=';')
    first_file=pd.concat([first_file,prepare_csv(x1)])
    rng=rng+1
    if rng>=15:
        break


In [18]:
first_file.values.flatten().shape

(10161600,)

In [8]:
X=first_file

In [9]:
X.tail()

,Longitude,Latitude,day_0,day_1,day_2,day_3,day_4,day_5,day_6,00:00:00-00:30:00,...,19:30:00-20:00:00,20:00:00-20:30:00,20:30:00-21:00:00,21:00:00-21:30:00,21:30:00-22:00:00,22:00:00-22:30:00,22:30:00-23:00:00,23:00:00-23:30:00,23:30:00-00:00:00,target
35035,2.2926,48.849157,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,0
35036,2.2926,48.849157,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,...,0,0,0,0,0,1,0,0,0,0
35037,2.2926,48.849157,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,1,0,0,0
35038,2.2926,48.849157,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
35039,2.2926,48.849157,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,1,0


In [21]:
for col in col_to_convert:
    X[col]=X[col].astype(int)

,mois,date,horodateur,Longitude,Latitude,day_of_week,tranche,nbr_place
0,1,2014-01-01,34870504,2.274419,48.866646,2,00:00:00-00:30:00,0
1,1,2014-01-02,34870504,2.274419,48.866646,3,00:00:00-00:30:00,2
2,1,2014-01-03,34870504,2.274419,48.866646,4,00:00:00-00:30:00,3
3,1,2014-01-04,34870504,2.274419,48.866646,5,00:00:00-00:30:00,8
4,1,2014-01-05,34870504,2.274419,48.866646,6,00:00:00-00:30:00,16
...,...,...,...,...,...,...,...,...
35035,12,2015-12-27,34870504,2.274419,48.866646,6,23:30:00-00:00:00,0
35036,12,2015-12-28,34870504,2.274419,48.866646,0,23:30:00-00:00:00,0
35037,12,2015-12-29,34870504,2.274419,48.866646,1,23:30:00-00:00:00,0
35038,12,2015-12-30,34870504,2.274419,48.866646,2,23:30:00-00:00:00,0


In [20]:
col_to_convert=first_file.columns.drop(['Longitude','Latitude','day_0','day_1','day_2','day_3','day_4','day_5','day_6','target']).tolist()

In [335]:
def Wajnberg_preparation_dataset(file):
    dftest=file
    dftest['mois']=pd.DatetimeIndex(dftest['date']).month
    del(dftest['date'])
    dftest=dftest.melt(id_vars=['mois','horodateur','Longitude','Latitude','day_of_week'],var_name='tranche',value_name='target')
    del (dftest['horodateur'])
    lb_make = LabelEncoder()
    dftest["tranche"] = lb_make.fit_transform(dftest["tranche"])
    return dftest

In [377]:
#open 5 files with new format:
rng=0
cols=Wajnberg_preparation_dataset(pd.read_csv(path+files[0],sep=';')).columns
first_file_wajn=pd.DataFrame([],columns=cols)

for e in files:
    x1=pd.read_csv(path+e,sep=';')
    first_file_wajn=pd.concat([first_file_wajn,Wajnberg_preparation_dataset(x1)])
    rng=rng+1
    if rng>=15:
        break


In [294]:
first_file_wajn

,mois,Longitude,Latitude,day_of_week,tranche,target
0,1,2.275394,48.867889,2,0,0
1,1,2.275394,48.867889,3,0,2
2,1,2.275394,48.867889,4,0,4
3,1,2.275394,48.867889,5,0,4
4,1,2.275394,48.867889,6,0,6
...,...,...,...,...,...,...
35035,12,2.273128,48.864807,6,47,0
35036,12,2.273128,48.864807,0,47,0
35037,12,2.273128,48.864807,1,47,0
35038,12,2.273128,48.864807,2,47,0


In [336]:
def makefilewajn(first_file):
    df1=pd.get_dummies(first_file['mois'],prefix='mois_')
    df2=pd.get_dummies(first_file['day_of_week'],prefix='jour_')
    df3=pd.get_dummies(first_file['tranche'],prefix='tranche_')
    ds=pd.concat([first_file.pop('target'),first_file.pop('Latitude'),first_file.pop('Longitude'),df1,df2,df3],axis=1)
    return ds

In [282]:
Train_data=ds

In [5]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [6]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.0.0


In [23]:
train_dataset = X.sample(frac=0.8,random_state=0)
test_dataset = X.drop(train_dataset.index)

In [283]:
#version normalisée:
train_dataset = Train_data.sample(frac=0.8,random_state=0)
test_dataset = Train_data.drop(train_dataset.index)

In [24]:
train_dataset

,Longitude,Latitude,day_0,day_1,day_2,day_3,day_4,day_5,day_6,00:00:00-00:30:00,...,19:30:00-20:00:00,20:00:00-20:30:00,20:30:00-21:00:00,21:00:00-21:30:00,21:30:00-22:00:00,22:00:00-22:30:00,22:30:00-23:00:00,23:00:00-23:30:00,23:30:00-00:00:00,target
174,2.310638,48.837340,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,...,0,0,0,0,0,0,0,0,0,6
15487,2.311226,48.838226,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,20
1541,2.316281,48.842897,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,...,0,0,0,0,0,0,0,0,0,17
13561,2.310314,48.836797,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,91
8128,2.273413,48.864937,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30051,2.310314,48.836797,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
17864,2.317050,48.843664,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,10
6353,2.292600,48.849157,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,52
20987,2.275394,48.867889,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,...,0,0,0,0,0,0,0,0,0,4


In [25]:
train_labels=train_dataset.pop('target')
test_labels=test_dataset.pop('target')


In [13]:
train_labels

174        6
15487     20
1541      17
13561     91
8128     141
        ... 
30051      0
17864     10
6353      52
20987      4
13324     89
Name: target, Length: 420480, dtype: object

In [26]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Longitude,420480.0,2.298720,0.018264,2.273128,2.274419,2.310638,2.313123,2.317050
Latitude,420480.0,48.849353,0.012173,48.836797,48.838786,48.842897,48.864937,48.867889
day_0,420480.0,0.142915,0.349987,0.000000,0.000000,0.000000,0.000000,1.000000
day_1,420480.0,0.142608,0.349674,0.000000,0.000000,0.000000,0.000000,1.000000
day_2,420480.0,0.143251,0.350329,0.000000,0.000000,0.000000,0.000000,1.000000
day_3,420480.0,0.144166,0.351259,0.000000,0.000000,0.000000,0.000000,1.000000
day_4,420480.0,0.142551,0.349615,0.000000,0.000000,0.000000,0.000000,1.000000
day_5,420480.0,0.142218,0.349274,0.000000,0.000000,0.000000,0.000000,1.000000
day_6,420480.0,0.142290,0.349348,0.000000,0.000000,0.000000,0.000000,1.000000
00:00:00-00:30:00,420480.0,0.020717,0.142435,0.000000,0.000000,0.000000,0.000000,1.000000


In [27]:
def build_model():
    inpt=len(train_dataset.keys())
    model = keras.Sequential([
    layers.Dense(inpt, activation='relu', input_shape=[inpt]),
    layers.Dense(inpt, activation='relu'),
    layers.Dense(1)
  ])

    optimizer = tf.keras.optimizers.RMSprop(0.03)

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    return model
    

In [28]:
model=build_model()

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 57)                3306      
_________________________________________________________________
dense_1 (Dense)              (None, 57)                3306      
_________________________________________________________________
dense_2 (Dense)              (None, 57)                3306      
_________________________________________________________________
dense_3 (Dense)              (None, 57)                3306      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 58        
Total params: 13,282
Trainable params: 13,282
Non-trainable params: 0
_________________________________________________________________


In [31]:
example_batch = test_dataset[:10]
example_result = model.predict(example_batch)
example_result

ValueError: Structure is a scalar but len(flat_sequence) == 0 > 1

In [164]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

1334       0
1633     107
1953       1
1998       1
6382       1
15794      4
19190     90
23008      4
27526      1
1334       0
Name: target, dtype: object

In [32]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto', 
                        restore_best_weights=True)

In [35]:
EPOCHS = 35
history = model.fit(train_dataset, train_labels,epochs=EPOCHS, validation_split = 0.2, verbose=2,callbacks=[early_stop,tfdocs.modeling.EpochDots()],batch_size=512)

Train on 336384 samples, validate on 84096 samples
Epoch 1/35

Epoch: 0, loss:1919.6853,  val_loss:1907.8475,  
.336384/336384 - 15s - loss: 1919.6853 - val_loss: 1907.8475
Epoch 2/35
.336384/336384 - 7s - loss: 1892.2515 - val_loss: 1900.7496
Epoch 3/35
.336384/336384 - 7s - loss: 1892.4898 - val_loss: 1898.9378
Epoch 4/35
.336384/336384 - 7s - loss: 1892.3304 - val_loss: 1899.8248
Epoch 5/35
.336384/336384 - 7s - loss: 1892.5427 - val_loss: 1898.6259
Epoch 6/35
.336384/336384 - 7s - loss: 1891.9173 - val_loss: 1901.3722
Epoch 7/35
.336384/336384 - 7s - loss: 1891.9028 - val_loss: 1900.6825
Epoch 8/35
.336384/336384 - 7s - loss: 1891.8542 - val_loss: 1899.3832
Epoch 9/35
.336384/336384 - 8s - loss: 1891.8301 - val_loss: 1900.6131
Epoch 10/35
.336384/336384 - 8s - loss: 1891.8973 - val_loss: 1901.5620
Epoch 11/35
.336384/336384 - 9s - loss: 1891.9457 - val_loss: 1898.3926
Epoch 12/35
.336384/336384 - 8s - loss: 1891.3732 - val_loss: 1903.8834
Epoch 13/35
.336384/336384 - 7s - loss: 189

In [36]:
hist          = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist

,loss,val_loss,epoch
0,1919.685274,1907.847516,0
1,1892.251488,1900.749611,1
2,1892.489811,1898.937772,2
3,1892.330374,1899.824790,3
4,1892.542749,1898.625936,4
5,1891.917310,1901.372215,5
6,1891.902801,1900.682501,6
7,1891.854163,1899.383218,7
8,1891.830092,1900.613121,8
9,1891.897267,1901.562024,9


In [37]:
def chart_regression(pred, y, sort=True):
    t = pd.DataFrame({'pred': pred.flatten(), 'y': y.values.flatten()})
    if sort:
        t.sort_values(by=['y'], inplace=True)
    plt.plot(t['y'].tolist(), label='expected')
    plt.plot(t['pred'].tolist(), label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

In [38]:
pred=model.predict(test_dataset)
chart_regression(pred,test_labels)

ValueError: Structure is a scalar but len(flat_sequence) == 0 > 1

In [40]:
for i in range(len(test_labels)):
    print (test_labels.values[i],pred.flatten()[i])

In [41]:

plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'bas': history}, metric = "accuracy")

KeyError: 'accuracy'

In [340]:
X.dtypes

day_0                 int32
day_1                 int32
day_2                 int32
day_3                 int32
day_4                 int32
day_5                 int32
day_6                 int32
00:00:00-00:30:00    object
00:30:00-01:00:00    object
01:00:00-01:30:00    object
01:30:00-02:00:00    object
02:00:00-02:30:00    object
02:30:00-03:00:00    object
03:00:00-03:30:00    object
03:30:00-04:00:00    object
04:00:00-04:30:00    object
04:30:00-05:00:00    object
05:00:00-05:30:00    object
05:30:00-06:00:00    object
06:00:00-06:30:00    object
06:30:00-07:00:00    object
07:00:00-07:30:00    object
07:30:00-08:00:00    object
08:00:00-08:30:00    object
08:30:00-09:00:00    object
09:00:00-09:30:00    object
09:30:00-10:00:00    object
10:00:00-10:30:00    object
10:30:00-11:00:00    object
11:00:00-11:30:00    object
11:30:00-12:00:00    object
12:00:00-12:30:00    object
12:30:00-13:00:00    object
13:00:00-13:30:00    object
13:30:00-14:00:00    object
14:00:00-14:30:00   

In [353]:
colstochange=X.columns.drop(['Latitude','Longitude']).tolist()
colstochange

['day_0',
 'day_1',
 'day_2',
 'day_3',
 'day_4',
 'day_5',
 'day_6',
 '00:00:00-00:30:00',
 '00:30:00-01:00:00',
 '01:00:00-01:30:00',
 '01:30:00-02:00:00',
 '02:00:00-02:30:00',
 '02:30:00-03:00:00',
 '03:00:00-03:30:00',
 '03:30:00-04:00:00',
 '04:00:00-04:30:00',
 '04:30:00-05:00:00',
 '05:00:00-05:30:00',
 '05:30:00-06:00:00',
 '06:00:00-06:30:00',
 '06:30:00-07:00:00',
 '07:00:00-07:30:00',
 '07:30:00-08:00:00',
 '08:00:00-08:30:00',
 '08:30:00-09:00:00',
 '09:00:00-09:30:00',
 '09:30:00-10:00:00',
 '10:00:00-10:30:00',
 '10:30:00-11:00:00',
 '11:00:00-11:30:00',
 '11:30:00-12:00:00',
 '12:00:00-12:30:00',
 '12:30:00-13:00:00',
 '13:00:00-13:30:00',
 '13:30:00-14:00:00',
 '14:00:00-14:30:00',
 '14:30:00-15:00:00',
 '15:00:00-15:30:00',
 '15:30:00-16:00:00',
 '16:00:00-16:30:00',
 '16:30:00-17:00:00',
 '17:00:00-17:30:00',
 '17:30:00-18:00:00',
 '18:00:00-18:30:00',
 '18:30:00-19:00:00',
 '19:00:00-19:30:00',
 '19:30:00-20:00:00',
 '20:00:00-20:30:00',
 '20:30:00-21:00:00',
 '21:0

In [355]:
for i in  colstochange :
    X[i]=X[i].astype(int)

In [358]:
X.shape

(175200, 58)

In [359]:
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# Convert to numpy - Classification
x_columns = X.columns.drop('target')
x = X[x_columns].values
y = X['target'].values

# Create train/test
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

# Build the neural network
model = Sequential()
model.add(Dense(58, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))# Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto', 
                        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
          callbacks=[monitor],verbose=2,epochs=1000,batch_size=512)

Train on 131400 samples, validate on 43800 samples
Epoch 1/1000
131400/131400 - 4s - loss: 3606.3602 - val_loss: 3118.5507
Epoch 2/1000
131400/131400 - 3s - loss: 3092.4558 - val_loss: 3118.1520
Epoch 3/1000
131400/131400 - 2s - loss: 3092.5746 - val_loss: 3119.6597
Epoch 4/1000
131400/131400 - 2s - loss: 3092.4731 - val_loss: 3118.3363
Epoch 5/1000
131400/131400 - 2s - loss: 3092.8371 - val_loss: 3118.3245
Epoch 6/1000
131400/131400 - 3s - loss: 3092.7759 - val_loss: 3118.2947
Epoch 7/1000
Restoring model weights from the end of the best epoch.
131400/131400 - 3s - loss: 3092.5484 - val_loss: 3118.4274
Epoch 00007: early stopping


In [365]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
Longitude,175200.0,2.274053,0.000802,2.273128,2.273413,2.273909,2.274419,2.275394
Latitude,175200.0,48.866047,0.001143,48.864807,48.864937,48.865955,48.866646,48.867889
day_0,175200.0,0.142466,0.349528,0.000000,0.000000,0.000000,0.000000,1.000000
day_1,175200.0,0.142466,0.349528,0.000000,0.000000,0.000000,0.000000,1.000000
day_2,175200.0,0.143836,0.350924,0.000000,0.000000,0.000000,0.000000,1.000000
day_3,175200.0,0.143836,0.350924,0.000000,0.000000,0.000000,0.000000,1.000000
day_4,175200.0,0.142466,0.349528,0.000000,0.000000,0.000000,0.000000,1.000000
day_5,175200.0,0.142466,0.349528,0.000000,0.000000,0.000000,0.000000,1.000000
day_6,175200.0,0.142466,0.349528,0.000000,0.000000,0.000000,0.000000,1.000000
00:00:00-00:30:00,175200.0,0.020833,0.142827,0.000000,0.000000,0.000000,0.000000,1.000000


In [32]:
from sklearn import metrics

# Predict
pred = model.predict(x_test)

# Measure MSE error.  
score = metrics.mean_squared_error(pred,y_test)
print("Final score (MSE): {}".format(score))

,id,job,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product
0,1,vv,c,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,b
1,2,kd,c,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,c
2,3,pe,c,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,b
3,4,11,c,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,b
4,5,kl,d,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,vv,c,51017.0,38.233333,1,5.454545,34,14.013489,41,0.881890,0.744094,0.104838,b
1996,1997,kl,d,26576.0,33.358333,2,3.632069,20,8.380497,38,0.944882,0.877953,0.063851,a
1997,1998,kl,d,28595.0,39.425000,3,7.168218,99,4.626950,36,0.759843,0.744094,0.098703,f
1998,1999,qp,c,67949.0,5.733333,0,8.936292,26,3.281439,46,0.909449,0.598425,0.117803,c


In [378]:
# Partie connection Mysql récupération du nombre de places:

In [386]:
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [383]:
conn= create_connection('parkindb.cxhhvpfp58jz.us-east-1.rds.amazonaws.com','user_parkin','User_ParkIN','ParkIN')

Connection to MySQL DB successful


In [387]:
query="select * from ParkIN.HORODATEURS_CAPACITY"
reponse = execute_read_query(conn, query)

In [412]:
champs=['horodateur','réelles','calculées','lat','long']
res=pd.DataFrame([],columns=champs)
for e in reponse:
    tmp=pd.DataFrame([e],columns=champs)
    res=pd.concat([res,tmp],axis=0,ignore_index=True)

In [413]:
res

,horodateur,réelles,calculées,lat,long
0,0,12,12,48.89400250,2.35407565
1,20301,17,17,48.85419900,2.33471000
2,40102,8,9,48.83038300,2.31482100
3,40202,12,12,48.82969700,2.31620000
4,40401,19,19,48.82884000,2.31826000
...,...,...,...,...,...
1036,13690202,3,3,48.87558500,2.34196600
1037,13710101,2,1,48.84301800,2.36323500
1038,13710203,18,18,48.81620458,2.34762595
1039,13710207,16,16,48.84174100,2.35836200
